In [14]:
#sf api setup

import time
import asyncio
import logging

from os.path import join

from sfapi_client         import Client, AsyncClient
from sfapi_client.compute import Machine
from sfapi_client.jobs    import JobState

from sfapi_connector import KeyManager, OsSFAPI, OsWrapper, LOGGER




In [15]:
# sf api send jobscript

N = 10000

target = "./sfapi_test"


    
job_script = f"""#!/bin/bash
#SBATCH -q debug
#SBATCH -A m669
#SBATCH -N 1
#SBATCH -n 32              # Number of tasks (64 tasks, 32 per node)
#SBATCH -C cpu
#SBATCH -t 00:05:00
#SBATCH -J sfapi-demo
#SBATCH --exclusive
#SBATCH --output={target}/sfapi-demo-%j.out
#SBATCH --error={target}/sfapi-demo-%j.error

# Print each command for debugging
set -x

# Load necessary modules, for example Python and Dask
module load python dask

echo "Starting scheduler..."

scheduler_file=$SCRATCH/scheduler_file.json
rm -f $scheduler_file

module load python

# Start scheduler
DASK_DISTRIBUTED__COMM__TIMEOUTS__CONNECT=3600s \
DASK_DISTRIBUTED__COMM__TIMEOUTS__TCP=3600s \
dask-scheduler \
    --interface hsn0 \
    --scheduler-file $scheduler_file &

dask_pid=$!

# Wait for the scheduler to start
sleep 5
until [ -f $scheduler_file ]
do
     echo "Waiting for scheduler to start..."
     sleep 5
done
echo "Scheduler started"

echo "Starting workers..."

# Start workers and redirect their output to a log file for better visibility
DASK_DISTRIBUTED__COMM__TIMEOUTS__CONNECT=3600s \
DASK_DISTRIBUTED__COMM__TIMEOUTS__TCP=3600s \
srun dask-worker \
    --scheduler-file $scheduler_file \
    --interface hsn0 \
    --nworkers 1 > $SCRATCH/worker_log.out 2>&1 &

echo "Workers started. Check $SCRATCH/worker_log.out for details."

# Wait a bit to ensure workers are started
echo "Sleeping..."
sleep 10

# Check number of workers
echo "Verifying number of Dask workers..."
python -c "from dask.distributed import Client; client = Client(scheduler_file='$scheduler_file'); print('Number of workers:', len(client.scheduler_info()['workers']))"

"""


# job_script = f"""#!/bin/bash
# #SBATCH -q debug
# #SBATCH -A m669
# #SBATCH -N 1
# #SBATCH -C cpu
# #SBATCH -t 00:01:00
# #SBATCH -J sfapi-demo
# #SBATCH --exclusive
# #SBATCH --output={target}/sfapi-demo-%j.out
# #SBATCH --error={target}/sfapi-demo-%j.error

# module load python
# # Prints N random numbers to form a normal distrobution
# python -c "import numpy as np; numbers = np.random.normal(size={N}); [print(n) for n in numbers]"
#     """ 

# os = OsWrapper()
# job_stript_path = join(target, "job_script.sh")

# with os.open(job_stript_path, "w", mk_target_dir=False) as f:
#     f.write(job_script)

km = KeyManager()
with Client(key=km.key) as client:
    perlmutter = client.compute(Machine.perlmutter)


    job = perlmutter.submit_job(job_script)
    print(f"Submitted_job: {job.jobid}")
    job_id = job.jobid

    while True:
        job.update()
        print(f"The job state is: {job.state} ({type(job.state)})")
        if job.state not in [JobState.PENDING, JobState.RUNNING, JobState.COMPLETING]:
            if job.state == JobState.FAILED:
                print("Job failed")
            elif job.state == JobState.COMPLETED:
                print("Job completed")
                output_file = perlmutter.ls(f"/global/homes/s/sanjeevc/sfapi_test/sfapi-demo-{job_id}.out")
                output_file = output_file[0]
                with output_file.open("r") as f:
                    print(f.read())
            elif job.state == JobState.TIMEOUT:
                print("Job timeout")
                output_file = perlmutter.ls(f"/global/homes/s/sanjeevc/sfapi_test/sfapi-demo-{job_id}.out")
                output_file = output_file[0]
                with output_file.open("r") as f:
                    print(f.read())    
            break
        try: 
            output_file = perlmutter.ls(f"/global/homes/s/sanjeevc/sfapi_test/sfapi-demo-{job_id}.out")
            output_file = output_file[0]
            with output_file.open("r") as f:
                print(f.read()) 
        except Exception as e:
            print(f"error: {e}")
        time.sleep(10)


In [7]:
output_file = perlmutter.ls(f"{target}/*.out")
output_file = output_file[0]
with output_file.open("r") as f:
    print(f.read())


RuntimeError: Cannot send a request, as the client has been closed.